In [ ]:
#miscellaneous libreries used 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 as cv #
from matplotlib import pyplot as plt
import sys
import random
from sklearn.neighbors import NearestNeighbors
import math

#Keras modules used
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import datasets
from keras.datasets import mnist
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, BatchNormalization
from keras.models import Model
from keras.callbacks import TensorBoard


In [ ]:
#defining what artists will be included in our database of images aswell as the sample size for each one
artists=["Vincent_van_Gogh","Pablo_Picasso","Hieronymus_Bosch","Giotto_di_Bondone","Salvador_Dali"]
sampleSize = 100
imgDatabase = []
#creation of the initial img database
for artist in artists:
    for i in range(sampleSize):
        img = cv.imread("../input/best-artworks-of-all-time/resized/resized/" + artist + "_" + str(i + 1) + ".jpg", 1)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        if img is None:
            sys.exit("Error. Image not found")
        imgDatabase.append([img,artist])

In [ ]:
# Displays a random image from the dataset
plt.imshow(imgDatabase[random.randint(1,(len(artists)*sampleSize))][0])
plt.show()

In [ ]:
# Counts how many images exist in a list per artist
def countImgsByArtist(imgSet):
    result = {}
    for (img,artist) in imgSet:
        result[artist] = result.get(artist, 0) +1
    print(result)

In [ ]:
#Prepars the images for the autoencoder by normalizing all values between 1 & 0 aswell as flatterning down the images 
def prepareImgs(imgSet):
    imgs = np.array([cv.resize(img[0].astype('float32') / 255,(width,height)) for img in imgSet])
    imgs2 = np.array([img.flatten() for img in imgs])
    return(imgs,imgs2)

In [ ]:
#shuffling the img database before performing a train test split on it
splitIndx = int(len(imgDatabase)*0.75)
random.shuffle(imgDatabase)

#setting the image size for the pictures
width = 100
height = 100
dim = width * height

# This is our input image for our autoencoders
input_img = keras.Input(shape=(dim*3,))

#the content and labels for the train set
train = imgDatabase[:splitIndx]
x_train2, x_train = prepareImgs(train)
y_train = [img[1] for img in train]

#the content and labels for the test set
test = imgDatabase[splitIndx:]
x_test2, x_test = prepareImgs(test)

y_test = [img[1] for img in test]

#How many images of each artist is in each set.
countImgsByArtist(train)
countImgsByArtist(test)
print(dim)

In [ ]:
# "encoded" is the encoded representation of the input
encoded = layers.Dense(32, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(dim*3, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder1 = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder1 = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(32,))

# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder1.layers[-1]

# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
#Compile and train the single node autoencoder
autoencoder1.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder1.fit(x_train, x_train,
                epochs=200,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Encode and decode the test images
encodedImgs1 = encoder1.predict(x_test)
decodedImgs1 = decoder.predict(encodedImgs1)

In [ ]:
#Display random examples of the denoised images from the autoencoders
def displayDenoiseResults(test, decodedImgs):
    n = 10  # How many Images that will display
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # Display original paintings
        ax = plt.subplot(1, n, i + 1)
        plt.imshow(test[i].reshape(width, height, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decodedImgs[i].reshape(width, height, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()


In [ ]:
displayDenoiseResults(x_test, decodedImgs1)

In [ ]:
#Our deep stacked autoencoder

# "encoded" is the encoded representation of the input
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

decoded = layers.Dense(dim*3, activation='sigmoid')(encoded)
# This model maps an input to its reconstruction
autoencoder2 = keras.Model(input_img, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder2 = keras.Model(input_img, encoded)

In [ ]:
# This is our encoded (32-dimensional) input
encoded_input2 = keras.Input(shape=(32,))
# Retrieve the last layer of the autoencoder model
decoder_layer2 = autoencoder2.layers[-1]
# Create the decoder model
decoder2 = keras.Model(encoded_input2, decoder_layer2(encoded_input2))

In [ ]:
#Compiles and train our deep stacked autoencoder
autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder2.fit(x_train, x_train,
                epochs=200,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Encode and decode the test images
encodedImgs2 = encoder2.predict(x_test)
decodedImgs2 = decoder2.predict(encodedImgs2)

In [ ]:
displayDenoiseResults(x_test,decodedImgs2)

In [ ]:
#Texture feature extraction by using gabor filters
ksize = 15
sigma = 4
theta = 2*np.pi/2
lamda = 1*np.pi/4
gamma=0.5
phi=0.7
kernel = cv.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv.CV_32F)

#creating the filters of the training and testing sets
plt.imshow(kernel)
train_fimgs = [cv.filter2D(cv.cvtColor(img.reshape(width, height, 3), cv.COLOR_BGR2GRAY), cv.CV_8UC3, kernel) for img in x_train]
test_fimgs = [cv.filter2D(cv.cvtColor(img.reshape(width, height, 3), cv.COLOR_BGR2GRAY), cv.CV_8UC3, kernel) for img in x_test]

In [ ]:
#Displays 5 Gabor Filter maps geenrated on top of 4 random samples from the training set.
for i in range(5):
    ax = plt.figure()
    ax.add_subplot(1,2, 1)
    plt.imshow(train_fimgs[i])
    ax.add_subplot(1,2, 2)
    plt.imshow(train[i][0])
    plt.show(block=True)

In [ ]:
#Creation of our Convolutional Autoencoder
input_img = keras.Input(shape=(width, height,3))
# "encoded" is the encoded representation of the input
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same', name = 'features')(x)#Layer given a name to access later on.

#"decoded" is the decoded representation of the created binary code from the encoder
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
# This model maps an input to its reconstruction

autoencoder3 = keras.Model(input_img, decoded)



In [ ]:
#Trains and compiles the convolutional autoencoder 
autoencoder3.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder3.fit(x_train2, x_train2,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test2, x_test2))

In [ ]:
#Extracts the features from the images of both the training and testing sets.
encoder3 = Model(inputs=autoencoder3.input, outputs=autoencoder3.get_layer('features').output)#grabs the final layer from the encoder component for feature extraction
testBinaryCodes = encoder3.predict(x_test2)
temp = testBinaryCodes.shape
decodedImgs4 = autoencoder3.predict(x_test2)

In [ ]:
n = 10  # How many Images that will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original paintings
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(decodedImgs4[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
#Setting the nearest neighbour objects for each classifier to compare agaisn't each test image.
#Setting the number of images we want returned 
kNeighbors = 5

#Fitting the train data features from the single node autoencoder
neighbor1 = NearestNeighbors(n_neighbors = kNeighbors)
neighbor1.fit(encoder1.predict(x_train))

#Fitting the train data features from the deep stacked autoencoder
neighbor2 = NearestNeighbors(n_neighbors = kNeighbors)
neighbor2.fit(encoder2.predict(x_train))

#Fitting the texture features from the gabour filter
neighbor3 = NearestNeighbors(n_neighbors = kNeighbors)
neighbor3.fit([fimg.flatten() for fimg in train_fimgs])

#Fitting the train data features from the convolutional autoencoder
neighbor4 = NearestNeighbors(n_neighbors = kNeighbors)
trainBinaryCodes = encoder3.predict(x_train2)
temp = trainBinaryCodes.shape
trainBinaryCodes = trainBinaryCodes.reshape(temp[0],(temp[1]*temp[2]*temp[3]))
neighbor4.fit(trainBinaryCodes)

In [ ]:
#calculating the accuracy of the CBIR with our deep stacked autoencoder
def cbirScore(nn,encodedImgs, test = test, y_train = y_train, y_test = y_test):
    score = []
    for i in range(len(test)):
        results = nn.kneighbors(encodedImgs[[i]])[1][0]
        query = y_test[i]
        temp = 0
        for pos in results:
            if(query == y_train[pos]):
                temp += 1
        score.append((temp/kNeighbors)*100)
    return(str(sum(score)/len(score)))

In [ ]:
#calculating the accuracy of the CBIR with Gabour Filters
def filterScore(nn,test_fimgs = test_fimgs, test = test, y_train = y_train, y_test = y_test):
    score = []
    for i in range(len(test)):
        results = nn.kneighbors([test_fimgs[indx].flatten()])[1][0]
        query = y_test[i]
        temp = 0
        for pos in results:
            if(query == y_train[pos]):
                temp += 1
        score.append((temp/kNeighbors)*100)
    return(str(sum(score)/len(score))) 

In [ ]:
#Displaying retrieved images for a query images
def displayQuerySample(results, indx, title, y_test = y_test, train = train):
    positions = results[1][0]
    plt.figure(figsize=(20,5))
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    ax = plt.subplot(1, kNeighbors+1, 1)
    fig = ax.get_figure()
    #Sets the first image in the figure as the query image
    plt.imshow(test[indx][0])
    ax.set_title("Query:\n" + y_test[indx])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    #Iterate through the retrieved images to add them to the figure
    for i in range(kNeighbors):
        img = train[positions[i]]
        ax = plt.subplot(1, kNeighbors+1, i + 2)
        plt.imshow(img[0])
        ax.set_title(img[1])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    positions = results[1][0]
    results

In [ ]:
#showing a sample of each classifier using the same query image
indx = random.randint(1,125)
#Sample of the single node autoencoder
displayQuerySample(neighbor1.kneighbors(encodedImgs1[[indx]]), indx, "Single Node Autoencoder")
print("Single Node Autoencoder Accuracy: " + cbirScore(neighbor1, encodedImgs1))
#Sample of the deep stacked auto encoder
displayQuerySample(neighbor2.kneighbors(encodedImgs2[[indx]]), indx, "Deep Stacked Autoencoder")
print("Deep Stacked Autoencoder Accuracy: " + cbirScore(neighbor2, encodedImgs2))
#Sample of the Gabor filter extraction
displayQuerySample(neighbor3.kneighbors([test_fimgs[indx].flatten()]), indx,"Gabor Filter Texture extraction")
print("Gabor Filter Texture extraction: " + filterScore(neighbor3))
#Sample of the convolutional autoencoder
testBinaryCodes = encoder3.predict(x_test2)
temp = testBinaryCodes.shape
testBinaryCodes = testBinaryCodes.reshape(temp[0],(temp[1]*temp[2]*temp[3]))
displayQuerySample(neighbor4.kneighbors(testBinaryCodes[[indx]]), indx, "Convolutional Autoencoder")
print("Convolutional Autoencoder: " + cbirScore(neighbor4, trainBinaryCodes))